In [5]:
# Extraccion de información desde la API de Yahoo Finance

!pip install yfinance --upgrade --no-cache-dir

In [2]:
# Listado de librerias requeridas

import yfinance as yf
import pandas as pd
import yfinance as yf
import time
try:
    from tabulate import tabulate
except ImportError:
    print("Por favor instalar tabulate: 'pip install tabulate' para obtener tabla organizada.")
    # Fallback en caso de que la libreria no esté instalada
    tabulate = None

In [3]:
# Definimos la lista de acciones.

lista_de_acciones = [
    'AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN',
    'QMMM', 'RGC', 'ALM', 'MU', 'HYMC',
    'FIEE', 'ANPA', 'SNDK', 'LIF', 'AMD'
]

# Lista para acumular la información de cada empresa.

datos_fundamentales = []

print("--- Iniciando descarga de información fundamental ---")

for ticker in lista_de_acciones:
    try:
        print(f"Procesando: {ticker}...")
        t = yf.Ticker(ticker)
        info = t.info

        # Añadimos el símbolo al diccionario para que sea una columna más.
        info['Ticker'] = ticker

        # Guardamos el diccionario completo en nuestra lista.
        datos_fundamentales.append(info)

    except Exception as e:
        print(f"  Error obteniendo datos para {ticker}: {e}")

# Convertimos la lista de diccionarios en un DataFrame.
df_info = pd.DataFrame(datos_fundamentales)

# Limpieza y Selección de Columnas:

if not df_info.empty:
    # Ponemos el Ticker como índice.
    df_info.set_index('Ticker', inplace=True)

    # Definimos columnas clave.

    columnas_deseadas = [
        'shortName', 'sector', 'industry', 'marketCap',
        'currentPrice', 'trailingPE', 'forwardPE',
        'dividendYield', 'beta', 'returnOnEquity'
    ]

    columnas_finales = df_info.columns.intersection(columnas_deseadas)
    df_resumen = df_info[columnas_finales].copy()

    nuevos_nombres = {
        'shortName': 'Nombre',
        'sector': 'Sector',
        'industry': 'Industria',
        'marketCap': 'Capitalización',
        'currentPrice': 'Precio Actual',
        'trailingPE': 'PE Actual',
        'forwardPE': 'PE Estimado',
        'dividendYield': 'Rendimiento Dividendo',
        'beta': 'Beta',
        'returnOnEquity': 'Retorno sobre Activo',

    }
    # Creamos una vista filtrada.
    df_resumen.rename(columns=nuevos_nombres, inplace=True)

    # --- Visualización ---
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)

    print("\n--- Tabla Resumen de Información Fundamental ---")
    print(df_resumen)

else:
    print("No se encontraron datos para ninguna acción de la lista.")


--- Iniciando descarga de información fundamental ---
Procesando: AAPL...
Procesando: MSFT...
Procesando: GOOGL...
Procesando: TSLA...
Procesando: AMZN...
Procesando: QMMM...
Procesando: RGC...
Procesando: ALM...
Procesando: MU...
Procesando: HYMC...
Procesando: FIEE...
Procesando: ANPA...
Procesando: SNDK...
Procesando: LIF...
Procesando: AMD...

--- Tabla Resumen de Información Fundamental ---
                                       Industria                  Sector  Rendimiento Dividendo   Beta    PE Actual  PE Estimado  Capitalización  Precio Actual  Retorno sobre Activo                           Nombre
Ticker                                                                                                                                                                                                                
AAPL                        Consumer Electronics              Technology                   0.41  1.093    34.621983    28.252178   3816436269056        258.280            

In [ ]:
total_info=len(df_info)
print(f"Total de filas en el DataFrame: {total_info}")

Total de filas en el DataFrame: 15


In [4]:
# Consolidación de datos (10 años).

lista_de_acciones = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'MU', 'AMD']

# --- Definición de la función de descarga ---
def descargar_datos_acciones(tickers_list):
    datos_portafolio = {}
    print(f"Iniciando descarga de {len(tickers_list)} activos...\n")

    for ticker in tickers_list:
        try:
            # Descargamos 10 años
            data = yf.download(ticker, period="10y", progress=False)

            if not data.empty:
                # Limpieza
                if isinstance(data.columns, pd.MultiIndex):
                    data.columns = data.columns.get_level_values(0)

                datos_portafolio[ticker] = data
                print(f"✔ {ticker}: {len(data)} registros descargados.")
            else:
                print(f"✖ {ticker}: No se encontraron datos (puede estar deslistada).")
        except Exception as e:
            print(f"Error al descargar {ticker}: {e}")

    return datos_portafolio

# Descargar
mis_datos = descargar_datos_acciones(lista_de_acciones)

# Vista Tabla
lista_completa_dfs = []

print("\nProcesando datos históricos...")

for ticker, df in mis_datos.items():
    if not df.empty:
        # Crea una copia
        df_temp = df.copy()

        # Agrega una columna con el nombre del Ticker
        df_temp['Ticker'] = ticker

        # Resetea el índice para que la 'Date' sea una columna normal y no el índice
        df_temp.reset_index(inplace=True)

        # Asegura el formato fecha
        df_temp['Date'] = df_temp['Date'].dt.date

        # Agregar a la lista maestra
        lista_completa_dfs.append(df_temp)

#DataFrame Final
if lista_completa_dfs:
    # Concatenamos todos los dataframes individuales en uno gigante
    df_final = pd.concat(lista_completa_dfs, ignore_index=True)

    #Remombre de Columnas
    nuevos_nombres = {
        'Date': 'Fecha',
        'Ticker': 'Simbolo',
        'Open': 'Apertura',
        'High': 'Maximo',
        'Low': 'Minimo',
        'Close': 'Cierre',
        'Adj Close': 'Cierre_Ajustado',
        'Volume': 'Volumen'
    }

    df_final = df_final.rename(columns=nuevos_nombres)
    # ----------------------------------------

    # Reordenar columnas
    columnas_orden = ['Simbolo', 'Fecha'] + [c for c in df_final.columns if c not in ['Simbolo', 'Fecha']]
    df_final = df_final[columnas_orden]

    print("\n--- Tabla Completa (10 años de historia) ---")
    print(f"Total de filas: {len(df_final)}")
    print(df_final.head()) # Muestra las primeras filas
    print("...")
    print(df_final.tail()) # Muestra las últimas filas
else:
    print("No se generaron datos para crear el DataFrame.")

Iniciando descarga de 7 activos...

✔ AAPL: 2514 registros descargados.
✔ MSFT: 2514 registros descargados.
✔ GOOGL: 2514 registros descargados.
✔ TSLA: 2514 registros descargados.
✔ AMZN: 2514 registros descargados.
✔ MU: 2514 registros descargados.
✔ AMD: 2514 registros descargados.

Procesando datos históricos...

--- Tabla Completa (10 años de historia) ---
Total de filas: 17598
Price Simbolo       Fecha     Cierre     Maximo     Minimo   Apertura    Volumen
0        AAPL  2016-02-01  21.741966  21.805097  21.509734  21.750985  163774000
1        AAPL  2016-02-02  21.302299  21.654030  21.257205  21.514239  149428800
2        AAPL  2016-02-03  21.723932  21.834412  21.212119  21.419550  183857200
3        AAPL  2016-02-04  21.898485  22.063971  21.578850  21.730733  185886800
4        AAPL  2016-02-05  21.313616  21.971024  21.238809  21.880347  185672400
...
Price Simbolo       Fecha      Cierre      Maximo      Minimo    Apertura   Volumen
17593     AMD  2026-01-23  259.679993  2

In [6]:
#Correctura de duplicados

print("\nDuplicados encontrados:", df_final.duplicated().sum())
df_final = df_final.drop_duplicates()


Duplicados encontrados: 0


In [7]:
# Convertir a tipo datetime

df_final['Fecha'] = pd.to_datetime(df_final['Fecha'], errors='coerce')

# Detectar valores inválidos (que no se pudieron convertir)
invalid_dates = df_final[df_final['Fecha'].isnull()]
print("Valores inválidos en 'Fecha':")
print(invalid_dates[['Fecha']])

# Rectificar cambio de tipo Object a datetime
df_final.info()

Valores inválidos en 'Fecha':
Empty DataFrame
Columns: [Fecha]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17598 entries, 0 to 17597
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Simbolo   17598 non-null  object        
 1   Fecha     17598 non-null  datetime64[ns]
 2   Cierre    17598 non-null  float64       
 3   Maximo    17598 non-null  float64       
 4   Minimo    17598 non-null  float64       
 5   Apertura  17598 non-null  float64       
 6   Volumen   17598 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 962.5+ KB


In [8]:
#Resumen infomracion financiera de marcas

lista_de_acciones = [
    'AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN',
    'QMMM', 'RGC', 'ALM', 'MU', 'HYMC',
    'FIEE', 'ANPA', 'SNDK', 'LIF', 'AMD'
]

# Lista para acumular la información de cada empresa
datos_fundamentales = []

print("--- Iniciando descarga de información fundamental ---")

for ticker in lista_de_acciones:
    try:
        t = yf.Ticker(ticker)
        info = t.info

        # Añadimos el símbolo al diccionario para identificar la fila
        info['Ticker'] = ticker

        datos_fundamentales.append(info)
        print(f"✓ {ticker}")

    except Exception as e:
        print(f"✕ Error en {ticker}: {e}")

# Convertimos la lista de diccionarios en un DataFrame
df_info = pd.DataFrame(datos_fundamentales)

# Limpieza y Selección de Columnas
if not df_info.empty:
    # Definimos columnas clave
    columnas_deseadas = [
        'Ticker', 'shortName', 'sector', 'industry', 'marketCap',
        'currentPrice', 'dividendYield'
    ]

    # Aseguramos que solo pedimos columnas que existen
    cols_existentes = [c for c in columnas_deseadas if c in df_info.columns]
    df_resumen = df_info[cols_existentes].copy()

    # --- FORMATO VISUAL

    # Rellenar datos vacíos con un guión '-'
    df_resumen = df_resumen.fillna('-')

    # Acortar nombres largos
    if 'shortName' in df_resumen.columns:
        df_resumen['shortName'] = df_resumen['shortName'].apply(lambda x: (x[:15] + '..') if isinstance(x, str) and len(x) > 15 else x)

    # Formatear números grandes
    # Convertimos Market Cap a Billones (B)
    if 'marketCap' in df_resumen.columns:
         df_resumen['marketCap'] = df_resumen['marketCap'].apply(
             lambda x: f"{x/1e9:.2f}B" if isinstance(x, (int, float)) else x
         )

    print("\n" * 2) # Espacio vertical
    print("--- TABLA RESUMEN DE MERCADO ---")

    # RENOMBRAR COLUMNAS ---
    nuevos_nombres = {
        'marketCap': 'Capitalización',
        'currentPrice': 'Precio Actual',
        'Ticker': 'Simbolo',
        'shortName': 'Nombre',
        'sector': 'Sector',
        'industry': 'Industria',
        'dividendYield': 'Rendimiento Dividendo'

    }

    # Aplicamos el cambio
    df_resumen = df_resumen.rename(columns=nuevos_nombres)
    # ----------------------------------------


    # Cambio del foramato de la tabla para mejor visualización
    if tabulate:
        # tablefmt='grid' crea los bordes estilo SQL/Excel
        # headers='keys' usa los nombres de las columnas
        print(tabulate(df_resumen, headers='keys', tablefmt='grid', showindex=False))
    else:
        # Si no tienes tabulate, usa el formato estándar de pandas
        pd.set_option('display.max_rows', None)
        pd.set_option('display.width', 1000)
        print(df_resumen)

else:
    print("No se encontraron datos para ninguna acción de la lista.")

--- Iniciando descarga de información fundamental ---
✓ AAPL
✓ MSFT
✓ GOOGL
✓ TSLA
✓ AMZN
✓ QMMM
✓ RGC
✓ ALM
✓ MU
✓ HYMC
✓ FIEE
✓ ANPA
✓ SNDK
✓ LIF
✓ AMD



--- TABLA RESUMEN DE MERCADO ---
+-----------+-------------------+------------------------+------------------------------------------+------------------+-----------------+-------------------------+
| Simbolo   | Nombre            | Sector                 | Industria                                | Capitalización   |   Precio Actual | Rendimiento Dividendo   |
+===========+===================+========================+==========================================+==================+=================+=========================+
| AAPL      | Apple Inc.        | Technology             | Consumer Electronics                     | 3816.44B         |         258.28  | 0.41                    |
+-----------+-------------------+------------------------+------------------------------------------+------------------+-----------------+------------

In [ ]:
# Exportación de datos CSV

nombre_csv = "historico_acciones_10y.csv"
df_final.to_csv(nombre_csv, index=False)
print(f"Archivo CSV guardado: {nombre_csv}")

Archivo CSV guardado: historico_acciones_10y.csv


In [ ]:
# Convertir a tipo datetime

df_final['Fecha'] = pd.to_datetime(df_final['Fecha'], errors='coerce')

# Detectar valores inválidos
invalid_dates = df_final[df_final['Fecha'].isnull()]
print("Valores inválidos en 'Fecha':")
print(invalid_dates[['Fecha']])

# Rectificar cambio de tipo Object a datetime
df_final.info()

Valores inválidos en 'Fecha':
Empty DataFrame
Columns: [Fecha]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17605 entries, 0 to 17604
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Simbolo   17605 non-null  object        
 1   Fecha     17605 non-null  datetime64[ns]
 2   Cierre    17605 non-null  float64       
 3   Maximo    17605 non-null  float64       
 4   Minimo    17605 non-null  float64       
 5   Apertura  17605 non-null  float64       
 6   Volumen   17605 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 962.9+ KB


In [ ]:
import yfinance as yf

# Crear un objeto Ticker para Microsoft(MSFT)
msft = yf.Ticker("MSFT")

# Obtener toda la información de la acción
print(msft.info)

# Obtener datos históricos de mercado del último mes
hist = msft.history(period="1mo")
print(hist)

# Obtener dividendos
print(msft.dividends)

# Obtener el balance general (balance sheet)
print(msft.balance_sheet)

{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The company's Productivity and Business Processes segment offers Microsoft 365 Commercial, Enterprise Mobility + Security, Windows Commercial, Power BI, Exchange, SharePoint, Microsoft Teams, Security and Compliance, and Copilot; Microsoft 365 Commercial products, such as Windows Commercial on-premises and Office licensed services; Microsoft 365 Consumer products and cloud services, such as Microsoft 365 Consumer subscriptions, Office licensed on-premises, and other consumer services;

In [ ]:
list(msft.info)

['address1',
 'city',
 'state',
 'zip',
 'country',
 'phone',
 'website',
 'industry',
 'industryKey',
 'industryDisp',
 'sector',
 'sectorKey',
 'sectorDisp',
 'longBusinessSummary',
 'fullTimeEmployees',
 'companyOfficers',
 'auditRisk',
 'boardRisk',
 'compensationRisk',
 'shareHolderRightsRisk',
 'overallRisk',
 'governanceEpochDate',
 'compensationAsOfEpochDate',
 'irWebsite',
 'executiveTeam',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'dividendRate',
 'dividendYield',
 'exDividendDate',
 'payoutRatio',
 'fiveYearAvgDividendYield',
 'beta',
 'trailingPE',
 'forwardPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'marketCap',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'allTimeHigh',
 'allTimeLow',
 'priceToSalesTrailing12Months',
 'fiftyDayAverage',